In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')

In [3]:
test = pd.read_csv('test.csv')

In [4]:
y = train['target']

In [5]:
train = train.drop('target', axis=1)

In [8]:
   
def preproc_data(df_input):
    df_output = df_input.copy()
    
    df_output = df_output.drop('_id', axis=1)
    


    month = list(set(df_output['month']))
    df_output['month'] = df_output.month.apply(lambda r: month.index(r))   
    


    df_output['duration_interrupted'] = df_output['duration'].apply(lambda d: 0 if d < 30 else d)
    
    df_output['nr.employed'] = df_output['nr.employed'].apply(lambda x: 1 if x <= 5087.65 else 0)
    
    df_output['default'] = df_output['default'].apply(lambda x: 0 if x == 'no' else 1)
    
    df_output['age_class'] = df_output['age'].apply(lambda x: 1 if (x >= 25 and x <  50) else 0)
    
    df_output = df_output.drop(['month', 'day_of_week', 'contact', 'pdays'], axis = 1)
    
    df_output['duration_class_my'] = df_output['duration'].apply(lambda x: 1 if (x >= 197 and x <= 700) else 0)

    df_output = pd.get_dummies(df_output, columns=[
         'marital',
         'education',
         'default',
         'housing',
         'loan',
       #  'contact',
         'poutcome' ,
        'job'
        #'month'
        
        
    ])
    
    return df_output

In [9]:
# train_work = preproc_data(train)
# test_work = preproc_data(test)

train_work = preproc_data(train)
test_work = preproc_data(test)

In [32]:
train_work.head(10)

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,54,87,1,999,0,1.4,93.444,-36.1,4.963,5228.1,...,0,0,0,0,0,0,0,0,1,0
1,36,291,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
2,53,182,1,999,0,1.4,93.444,-36.1,4.965,5228.1,...,0,0,0,0,0,0,1,0,0,0
3,34,180,2,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,0,0,0,1
4,29,6,1,999,0,-1.7,94.027,-38.3,0.890,4991.6,...,0,0,0,0,0,1,0,0,0,0
5,56,64,2,999,0,1.4,94.465,-41.8,4.961,5228.1,...,0,0,0,0,0,0,0,1,0,0
6,33,312,2,999,0,-1.8,93.876,-40.0,0.685,5008.7,...,0,1,0,0,0,0,1,0,0,0
7,36,20,1,999,1,-0.1,93.200,-42.0,4.120,5195.8,...,0,0,1,0,0,0,0,0,0,1
8,70,585,1,6,3,-1.1,94.601,-49.5,0.987,4963.6,...,0,0,0,1,0,0,0,1,0,0
9,32,196,2,999,1,-1.8,92.893,-46.2,1.327,5099.1,...,0,1,0,0,0,0,0,1,0,0


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train_work, y, test_size=0.33, random_state=42)

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

# classifiers = {
#     'tree': DecisionTreeClassifier(max_depth=5), 
#     'logistic': LogisticRegression(),
#     'forest': RandomForestClassifier(),
#     'kn': KNeighborsClassifier(n_neighbors=10)
# }

In [15]:
import numpy as np
dtc = DecisionTreeClassifier(max_depth=10)
dtc.fit(X_train, y_train)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

lr = LogisticRegression()
#lr = LogisticRegression(n_jobs=-1, class_weight='balanced', solver='newton-cg', multi_class='multinomial')
lr.fit(X_train, y_train)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, train_work.columns[indices[f]], importances[indices[f]]))

Feature ranking:
1. feature duration (0.170716)
2. feature duration_interrupted (0.167100)
3. feature euribor3m (0.107198)
4. feature age (0.068185)
5. feature nr.employed (0.058583)
6. feature cons.conf.idx (0.038802)
7. feature cons.price.idx (0.038683)
8. feature poutcome_success (0.034828)
9. feature campaign (0.032278)
10. feature emp.var.rate (0.031949)
11. feature duration_class_my (0.019492)
12. feature previous (0.013046)
13. feature housing_yes (0.012070)
14. feature education_university.degree (0.011707)
15. feature housing_no (0.011441)
16. feature job_admin. (0.011359)
17. feature marital_married (0.010527)
18. feature education_high.school (0.010263)
19. feature marital_single (0.009507)
20. feature age_class (0.009475)
21. feature poutcome_failure (0.009236)
22. feature loan_no (0.008414)
23. feature job_technician (0.008351)
24. feature loan_yes (0.008012)
25. feature job_blue-collar (0.007735)
26. feature poutcome_nonexistent (0.007654)
27. feature education_profession

In [16]:
roc_auc_score(y_test, dtc.predict_proba(X_test)[:,1])

0.8462023101940114

In [17]:
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])

0.9288339127509252

In [18]:
roc_auc_score(y_test, knn.predict_proba(X_test)[:,1])

0.7834122088893875

In [19]:
from sklearn.metrics import log_loss
log_loss(y_test, dtc.predict_proba(X_test)[:,1])

1.1524042093425304

In [20]:
log_loss(y_test, lr.predict_proba(X_test)[:,1])

0.21230495041632705

In [21]:
log_loss(y_test, knn.predict_proba(X_test)[:,1])

1.5940843728644436

In [22]:
train_work_tmp = train_work.copy()
train_work_tmp['dtc'] = dtc.predict_proba(train_work)[:,1]


train_work_tmp['rf'] = rf.predict_proba(train_work)[:,1]
train_work = train_work_tmp

test_work_tmp = test_work.copy()
test_work_tmp['dtc'] = dtc.predict_proba(test_work)[:,1]


test_work_tmp['rf'] = rf.predict_proba(test_work)[:,1]
test_work = test_work_tmp

In [23]:
X_train, X_test, y_train, y_test = train_test_split(train_work, y, test_size=0.33, random_state=42)
lr.fit(X_train, y_train)
roc_auc_score(y_test, lr.predict_proba(X_test)[:,1])

0.9360657919330118

In [24]:
result_predict = pd.DataFrame(data={'_id': test['_id'], 'target': lr.predict_proba(test_work)[:,1]})

In [25]:
result_predict_test = pd.DataFrame(data={'_id': test['_id'], 'target': lr.predict(test_work)})

In [26]:
result_predict_test[result_predict_test['target'] == 1]['target'].agg('count')

1599

In [ ]:
# del result_predict

In [77]:
result_predict.to_csv('submit_lr.csv', index=False)